TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Feb 17 13:10:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   53C    P8     6W /  N/A |    331MiB /  6144MiB |     26%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 17.0 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [3]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
import random

In [68]:
# # Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
tag_name = 'science'

lower_corpus = False

if lower_corpus:
  corpus_file_name = tag_name + '_lowercase'
else:
  corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'

In [6]:
corpus_file_name

'science_sentence_case'

In [7]:
# if on Google Colab, load transcripts file.
# with open(f'/content/drive/MyDrive/ted/{corpus_file_name}.txt') as f:
#   corpus = f.read()
# f.close()

# if run locally:
with open(f'{root_path}{corpus_file_name}.txt') as f:
  corpus = f.read()
f.close()

In [8]:
corpus[0:500]

"So I want to start by offering you a free no-tech life hack, and all it requires of you is  that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out.  I see you. So I want you to pay atten"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [9]:
chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 77


In [10]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '"': 2, '%': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ';': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'X': 44, 'Y': 45, '[': 46, ']': 47, 'a': 48, 'b': 49, 'c': 50, 'd': 51, 'e': 52, 'f': 53, 'g': 54, 'h': 55, 'i': 56, 'j': 57, 'k': 58, 'l': 59, 'm': 60, 'n': 61, 'o': 62, 'p': 63, 'q': 64, 'r': 65, 's': 66, 't': 67, 'u': 68, 'v': 69, 'w': 70, 'x': 71, 'y': 72, 'z': 73, 'â': 74, '”': 75, '€': 76}


In [11]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '"', 3: '%', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '0', 10: '1', 11: '2', 12: '3', 13: '4', 14: '5', 15: '6', 16: '7', 17: '8', 18: '9', 19: ';', 20: '?', 21: 'A', 22: 'B', 23: 'C', 24: 'D', 25: 'E', 26: 'F', 27: 'G', 28: 'H', 29: 'I', 30: 'J', 31: 'K', 32: 'L', 33: 'M', 34: 'N', 35: 'O', 36: 'P', 37: 'Q', 38: 'R', 39: 'S', 40: 'T', 41: 'U', 42: 'V', 43: 'W', 44: 'X', 45: 'Y', 46: '[', 47: ']', 48: 'a', 49: 'b', 50: 'c', 51: 'd', 52: 'e', 53: 'f', 54: 'g', 55: 'h', 56: 'i', 57: 'j', 58: 'k', 59: 'l', 60: 'm', 61: 'n', 62: 'o', 63: 'p', 64: 'q', 65: 'r', 66: 's', 67: 't', 68: 'u', 69: 'v', 70: 'w', 71: 'x', 72: 'y', 73: 'z', 74: 'â', 75: '”', 76: '€'}


In [12]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 80,583 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

In [13]:
sequence_length = 50

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [14]:
corpus[0:50]

'So I want to start by offering you a free no-tech '

In [15]:
corpus[1:41]

'o I want to start by offering you a free'

In [16]:
corpus[41]

' '

In [17]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [18]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[39, 62, 0, 29, 0, 70, 48, 61, 67, 0, 67, 62, 0, 66, 67, 48, 65, 67, 0, 49, 72, 0, 62, 53, 53, 52, 65, 56, 61, 54, 0, 72, 62, 68, 0, 48, 0, 53, 65, 52, 52, 0, 61, 62, 7, 67, 52, 50, 55, 0]
[62, 0, 29, 0, 70, 48, 61, 67, 0, 67, 62, 0, 66, 67, 48, 65, 67, 0, 49, 72, 0, 62, 53, 53, 52, 65, 56, 61, 54, 0, 72, 62, 68, 0, 48, 0, 53, 65, 52, 52, 0, 61, 62, 7, 67, 52, 50, 55, 0, 59]


In [19]:
print ( y_numeric_list[0:2])

[59, 56]


In [20]:
indices_char[51]

'd'

In [21]:
corpus[0:43]

'So I want to start by offering you a free n'

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [22]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 80,533 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [32]:
print(X_numeric_list[0])

[39, 62, 0, 29, 0, 70, 48, 61, 67, 0, 67, 62, 0, 66, 67, 48, 65, 67, 0, 49, 72, 0, 62, 53, 53, 52, 65, 56, 61, 54, 0, 72, 62, 68, 0, 48, 0, 53, 65, 52, 52, 0, 61, 62, 7, 67, 52, 50, 55, 0]


In [38]:
len(X_numeric_list)

80533

In [39]:
sequence_length

50

In [23]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [43]:
39/77

0.5064935064935064

In [42]:
len(char_indices)

77

In [46]:
X[0][0]

array([0.50649351])

In [24]:
X.shape

(80533, 50, 1)

In [25]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [26]:
# Look at the first value of X
X[0]

array([[0.50649351],
       [0.80519481],
       [0.        ],
       [0.37662338],
       [0.        ],
       [0.90909091],
       [0.62337662],
       [0.79220779],
       [0.87012987],
       [0.        ],
       [0.87012987],
       [0.80519481],
       [0.        ],
       [0.85714286],
       [0.87012987],
       [0.62337662],
       [0.84415584],
       [0.87012987],
       [0.        ],
       [0.63636364],
       [0.93506494],
       [0.        ],
       [0.80519481],
       [0.68831169],
       [0.68831169],
       [0.67532468],
       [0.84415584],
       [0.72727273],
       [0.79220779],
       [0.7012987 ],
       [0.        ],
       [0.93506494],
       [0.80519481],
       [0.88311688],
       [0.        ],
       [0.62337662],
       [0.        ],
       [0.68831169],
       [0.84415584],
       [0.67532468],
       [0.67532468],
       [0.        ],
       [0.79220779],
       [0.80519481],
       [0.09090909],
       [0.87012987],
       [0.67532468],
       [0.649

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [27]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

---
### Create LSTM Model

This [blog post](https://towardsdatascience.com/choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras-f8e9ed76f046) has a rule of thumb of how many nodes to include in an LSTM.

In [92]:
hidden_nodes = int(2/3 * (len(chars) * sequence_length))
print(f"The number of hidden nodes is {hidden_nodes}.")

The number of hidden nodes is 2566.


Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [93]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = f'{root_path}{corpus_file_name}/{corpus_file_name}.ckpt'
checkpoint_dir = f'{root_path}{corpus_file_name}/'

batch_size = 1

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_freq=5*batch_size)

In [28]:
X.shape[1]

50

In [29]:
X.shape[2]

1

In [94]:
# define the LSTM model
model = Sequential()
model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
model.fit(X, y, epochs=50, batch_size=200, callbacks=[cp_callback])

Epoch 1/50
  5/403 [..............................] - ETA: 14:39 - loss: 4.1348
Epoch 00001: saving model to ../data/science_sentence_case\science_sentence_case.ckpt


INFO:tensorflow:Assets written to: ../data/science_sentence_case\science_sentence_case.ckpt\assets


INFO:tensorflow:Assets written to: ../data/science_sentence_case\science_sentence_case.ckpt\assets


 10/403 [..............................] - ETA: 25:12 - loss: 3.9247
Epoch 00001: saving model to ../data/science_sentence_case\science_sentence_case.ckpt


INFO:tensorflow:Assets written to: ../data/science_sentence_case\science_sentence_case.ckpt\assets


...

INFO:tensorflow:Assets written to: ../data/science_sentence_case\science_sentence_case.ckpt\assets


 34/403 [=>............................] - ETA: 36:02 - loss: 2.5792
Epoch 00008: saving model to ../data/science_sentence_case\science_sentence_case.ckpt


INFO:tensorflow:Assets written to: ../data/science_sentence_case\science_sentence_case.ckpt\assets


INFO:tensorflow:Assets written to: ../data/science_sentence_case\science_sentence_case.ckpt\assets


 36/403 [=>............................] - ETA: 38:02 - loss: 2.5795

In [ ]:
# Save the model to my Google Drive so I can load it later from another notebook
#model.save(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

model.save(f'{root_path}/ted_model_{tag_name}')